In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from math import factorial

import json
import re

import torch
from torch import nn
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoConfig

from datasets import Dataset
from transformers import AutoModelForCausalLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
from peft import LoraConfig, get_peft_model

import sys
from eval_utils import get_metrics_computer, PrintCallback
from utils import get_preprocessor, Format, ShuffleCollator, count_parameters

/home/jupyter/.local/lib/python3.10/site-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
2024-05-15 10:28:53.904787: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
device

'cuda:0'

In [3]:
with open("configs/config_ruT5-base-st.json", "rb") as config:
    params = json.load(config)

params

{'format': 'SpecTokens',
 'max_bundles': 5,
 'model': 'ai-forever/ruT5-base',
 'add_nl_token': False,
 'add_eos_token': False,
 'change_pad_to_eos': False,
 'shuffle_bundles': True,
 'save_folder': 'ruT5-base',
 'train': {'n_epochs': 10,
  'lr': 5e-05,
  'batch_size': 16,
  'weight_decay': 0.01,
  'scheduler': 'cosine',
  'warmup_steps': 500,
  'fp16': True},
 'eval': {'batch_size': 16, 'show': 5}}

In [4]:
out_format = Format.SpecTokens if params["format"] == "SpecTokens" else Format.JustJson

In [5]:
train_data = pd.read_csv("~/work/resources/data/ads_train.csv")
train_data = train_data[train_data["n_bundles"] <= params.get("max_bundles", np.inf)]
train_data.set_index(np.arange(len(train_data)), inplace=True)
print(f"We have train datset of size {len(train_data)}")
train_data.head()

We have train datset of size 8632


,Text,bundles,n_bundles
0,Продам телефон POCO F4 128GB телефону 8 месяце...,"[{""Title"": ""\u0442\u0435\u043b\u0435\u0444\u04...",1
1,"продам sup борд 10'6 - 320х76х15 см, пользова...","[{""Title"": ""sup \u0431\u043e\u0440\u0434 \u043...",1
2,Изменились планы . Отдам за 2000,[],0
3,Продам рацию MegaJet MJ-333 в хорошем рабочем ...,"[{""Title"": ""\u0440\u0430\u0434\u0438\u043e\u04...",1
4,Продам PlayStation 3\n3 джойстик и все диски\n...,"[{""Title"": ""PlayStation 3 \u0441 3 \u0434\u043...",1


In [6]:
eval_data = pd.read_csv("~/work/resources/data/ads_eval.csv")
eval_data = eval_data[:params.get("max_eval_size", len(eval_data))]
eval_data.set_index(np.arange(len(eval_data)), inplace=True)
print(f"We have eval datset of size {len(eval_data)}")
eval_data.head()

We have eval datset of size 611


,Text,bundles,n_bundles
0,Распродажа \nПлатья \nЦена: 300р\nРазмер: 44-4...,"[{""Title"": ""\u041f\u043b\u0430\u0442\u044c\u04...",1
1,"вегетарианская энциклопедия вкусов, что с чем ...","[{""Title"": ""\u0432\u0435\u0433\u0435\u0442\u04...",1
2,"продам щетку для сухого массажа, новая (так и ...","[{""Title"": ""\u0429\u0435\u0442\u043a\u0430 \u0...",1
3,"Зимний тёплый комбинезон, пух, Crockid, p.74...","[{""Title"": ""\u0417\u0438\u043c\u043d\u0438\u04...",1
4,"удлинитель икеа 1,5 м на 6 розеток\n25 лари\n📍...","[{""Title"": ""\u0443\u0434\u043b\u0438\u043d\u04...",1


In [7]:
model_checkpoint = params["model"]
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

tokenizer.add_tokens(["₾", "$", "€"], special_tokens=False)
if params.get("add_nl_token", False):
    tokenizer.add_tokens(["<NL>"], special_tokens=False)
if out_format == Format.SpecTokens:
    tokenizer.add_tokens(["<BOB>", "<EOB>", "<BOT>", "<EOT>", "<BOP>", "<EOP>", "<BOC1>", "<EOC1>", "<BOC2>", "<EOC2>"], special_tokens=False)
elif out_format == Format.JustJson:
    tokenizer.add_tokens(["{", "}"], special_tokens=False)
model.resize_token_embeddings(len(tokenizer))

/home/jupyter/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.10/dist-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in th

Embedding(32112, 768)

In [8]:
train_dataset = Dataset.from_pandas(train_data[["Text", "bundles"]])
train_ads = train_dataset.map(
    get_preprocessor(tokenizer, out_format),
    batched=True,
    num_proc=4,
    remove_columns=train_dataset.column_names
)
train_ads = train_ads.flatten()

eval_dataset = Dataset.from_pandas(eval_data[["Text", "bundles"]])
eval_ads = eval_dataset.map(
    get_preprocessor(tokenizer, out_format),
    batched=True,
    num_proc=4,
    remove_columns=eval_dataset.column_names
)
eval_ads = eval_ads.flatten()

Map (num_proc=4): 100%|██████████| 611/611 [00:00<00:00, 1290.79 examples/s]


In [9]:
if params.get("change_pad_to_eos", False):
    tokenizer.pad_token = tokenizer.eos_token

data_collator = ShuffleCollator(tokenizer, out_format, params.get("add_eos_token", False), params.get("shuffle_bundles", False))

In [10]:
train_params = params["train"]
eval_params = params["eval"]

lora_rank = params.get("lora", {}).get("rank")
if lora_rank is not None:
    lora_config = LoraConfig(
        r=lora_rank,
        lora_alpha=32,
        target_modules=["k", "q", "v", "o", "lm_head"],
        lora_dropout=0.05,
        bias="none",
        task_type="SEQ_2_SEQ_LM",
    )

    model = get_peft_model(model, lora_config)

count_parameters(model)

training_args = Seq2SeqTrainingArguments(
    output_dir="tmp_checkpoints",
    overwrite_output_dir=True,
    evaluation_strategy="epoch",
    learning_rate=train_params["lr"],
    per_device_train_batch_size=train_params["batch_size"],
    per_device_eval_batch_size=eval_params["batch_size"],
    weight_decay=train_params.get("weight_decay", 0),
    save_total_limit=1,
    num_train_epochs=train_params["n_epochs"],
    predict_with_generate=True,
    generation_max_length=256,
    lr_scheduler_type=train_params.get("scheduler", "cosine"),
    group_by_length=False,
    warmup_steps=train_params.get("warmup_steps", 0),
    fp16=train_params.get("fp16", False),
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_ads,
    eval_dataset=eval_ads,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=get_metrics_computer(tokenizer, out_format),
    callbacks=[PrintCallback(out_format, show=eval_params.get("show", 3), device=device)]
)

Total parameters: 222,891,264
Trainable parameters: 222,891,264
Non-trainable parameters: 0


In [ ]:
trainer.train()

  9%|▉         | 500/5400 [01:55<18:46,  4.35it/s]

{'loss': 2.3505, 'grad_norm': 0.9308339357376099, 'learning_rate': 4.92e-05, 'epoch': 0.93}


100%|██████████| 39/39 [02:47<00:00,  1.93s/it]



                                                  
100%|██████████| 39/39 [14:44<00:00,  1.93s/it]

{'eval_loss': 0.12973809242248535, 'eval_Global BLEU': 61.197050713994074, 'eval_valid_answer_structure_precent': 95.34109816971714, 'eval_valid_bundles_precent': 100.0, 'eval_n_bundles_mae': 0.3787085514834206, 'eval_mean_over_bundles': 0.06806282722513089, 'eval_mean_under_bundles': 0.3106457242582897, 'eval_title_bleu_1_bundle': 31.959071138182768, 'eval_title_chrf_1_bundle': 69.25242721199052, 'eval_price_match_precent_1_bundle': 84.28030303030303, 'eval_currency_match_precent_1_bundle': 92.23484848484848, 'eval_count_match_precent_1_bundle': 89.01515151515152, 'eval_title_bleu_multi_bundle': 19.27024936521683, 'eval_title_chrf_multi_bundle': 49.01640137530453, 'eval_price_match_precent_multi_bundle': 33.33333333333333, 'eval_currency_match_precent_multi_bundle': 83.33333333333334, 'eval_count_match_precent_multi_bundle': 100.0, 'eval_runtime': 886.8077, 'eval_samples_per_second': 0.689, 'eval_steps_per_second': 0.044, 'epoch': 1.0}



ORIGINAL TEXT
	RAW
	 Распродажа Платья Цена: 300р Размер: 44-46-48-50 Без выбора цвета Арт: 2Г-14 корпус Б</s>

TARGET
	RAW
	 <BOB> <BOT> Платья размер 44-46-48-50 <EOT> <BOP> 300 <EOP> <BOC1> RUB <EOC1> <BOC2> 1 <EOC2> <EOB></s>
	DECODED:
	 [{'title': 'Платья размер 44-46-48-50', 'price': '300', 'currency': 'RUB', 'count': '1'}]

PREDICTED
	RAW
	 <BOB> <BOT> Платья <EOT> <BOP> 300 <EOP> <BOC1> RUB <EOC1> <BOC2> 1 <EOC2> <EOB></s>
	DECODED:
	 [{'title': 'Платья', 'price': '300', 'currency': 'RUB', 'count': '1'}]
--------------------------------------------------


ORIGINAL TEXT
	RAW
	 вегетарианская энциклопедия вкусов, что с чем сочетается, справочник от шеф поваров. 15 eur. лимассол.</s>

TARGET
	RAW
	 <BOB> <BOT> вегетарианская энциклопедия вкусов, что с чем сочетается, справочник от шеф поваров <EOT> <BOP> 15 <EOP> <BOC1> EUR <EOC1> <BOC2> 1 <EOC2> <EOB></s>
	DECODED:
	 [{'title': 'вегетарианская энциклопедия вкусов, что с чем сочетается, справочник от шеф поваров', 'price': '15',

...

{'eval_loss': 0.09591393172740936, 'eval_Global BLEU': 77.593588620695, 'eval_valid_answer_structure_precent': 99.0, 'eval_valid_bundles_precent': 100.0, 'eval_n_bundles_mae': 0.19023569023569023, 'eval_mean_over_bundles': 0.13973063973063973, 'eval_mean_under_bundles': 0.050505050505050504, 'eval_title_bleu_1_bundle': 36.03530959297528, 'eval_title_chrf_1_bundle': 72.44280639645692, 'eval_price_match_precent_1_bundle': 93.34763948497854, 'eval_currency_match_precent_1_bundle': 94.63519313304721, 'eval_count_match_precent_1_bundle': 91.41630901287554, 'eval_title_bleu_multi_bundle': 25.831816699965792, 'eval_title_chrf_multi_bundle': 60.79868806603791, 'eval_price_match_precent_multi_bundle': 63.789682539682545, 'eval_currency_match_precent_multi_bundle': 90.47619047619048, 'eval_count_match_precent_multi_bundle': 92.81746031746032, 'eval_runtime': 1058.458, 'eval_samples_per_second': 0.577, 'eval_steps_per_second': 0.037, 'epoch': 3.0}



ORIGINAL TEXT
	RAW
	 Распродажа Платья Цена: 300р Размер: 44-46-48-50 Без выбора цвета Арт: 2Г-14 корпус Б</s>

TARGET
	RAW
	 <BOB> <BOT> Платья размер 44-46-48-50 <EOT> <BOP> 300 <EOP> <BOC1> RUB <EOC1> <BOC2> 1 <EOC2> <EOB></s>
	DECODED:
	 [{'title': 'Платья размер 44-46-48-50', 'price': '300', 'currency': 'RUB', 'count': '1'}]

PREDICTED
	RAW
	 <BOB> <BOT> Платья <EOT> <BOP> 300 <EOP> <BOC1> RUB <EOC1> <BOC2> 1 <EOC2> <EOB></s>
	DECODED:
	 [{'title': 'Платья', 'price': '300', 'currency': 'RUB', 'count': '1'}]
--------------------------------------------------


ORIGINAL TEXT
	RAW
	 вегетарианская энциклопедия вкусов, что с чем сочетается, справочник от шеф поваров. 15 eur. лимассол.</s>

TARGET
	RAW
	 <BOB> <BOT> вегетарианская энциклопедия вкусов, что с чем сочетается, справочник от шеф поваров <EOT> <BOP> 15 <EOP> <BOC1> EUR <EOC1> <BOC2> 1 <EOC2> <EOB></s>
	DECODED:
	 [{'title': 'вегетарианская энциклопедия вкусов, что с чем сочетается, справочник от шеф поваров', 'price': '15',

 37%|███▋      | 2000/5400 [59:15<13:12,  4.29it/s]     

{'loss': 0.1126, 'grad_norm': 1.0584112405776978, 'learning_rate': 3.940789717249119e-05, 'epoch': 3.7}


100%|██████████| 39/39 [03:06<00:00,  4.05s/it]

In [ ]:
if lora_rank is not None:
    model = model.merge_and_unload()
output_dir = f"../good_checkpoints/{params["save_folder"]}"
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)